# IMPORT

In [ ]:
import os
import joblib

import pandas as pd
from tqdm import tqdm
from transformers import pipeline

# FUNCTION

In [21]:
def sentiment_analysis(pipe, texts):
    label2id = {"negative": 0, "positive": 1, "neutral": 2}
    max_length = 512
    results = pipe(
        texts,
        truncation=True,
        max_length=max_length,
        padding=True,
        batch_size=16
    )
    labels = [item["label"] for item in results]
    scores = [item["score"] for item in results]
    
    label_ids = [label2id[label] for label in labels]
    return label_ids

In [10]:
def load_json(path):
    with open(path, "r", encoding="utf-8") as file:
        json_content = json.load(file)
        return json_content
    
#---- Save json
def save_json(path, content):
    with open(path, "w", encoding="utf-8") as file:
        json.dump(content, file, ensure_ascii=False, indent=3)
        
        
#---- Load numpy
def load_npy(path):
    npy_file = np.load(path, allow_pickle=True)
    if hasattr(npy_file, 'item') and npy_file.size == 1:
        return npy_file.item()
    return npy_file


# LOAD DATA

In [4]:
test_path = r"C:\APAC\all_projects\finetuning-airflow-project\project_2_training\data\test.csv"
test_df = pd.read_csv(test_path)

In [23]:
texts = list(test_df["title"].values)

# BASELINE FINBERT

In [18]:
pipe = pipeline("text-classification", model="ProsusAI/finbert")      

Device set to use cpu


In [25]:
batch_size = 4
finbert_preds = []
for start in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[start: start+batch_size]
    batch_preds = sentiment_analysis(pipe, batch_texts)
    finbert_preds.extend(batch_preds)

100%|██████████| 243/243 [00:51<00:00,  4.74it/s]
